In [1]:
import numpy as np
import scipy.integrate

from pyquad import quad, quad_grid
from jit import jit_integrand

In [2]:
def test_integrand_func(x, alpha, beta, i, j, k, l):                                             
    return x * alpha * beta + i * j * k     

# pyquad.quad <-> scipy.integrate.quad

In a basic sense, it is possible to just drop-in the `pyquad.quad` in place of `scipy.integrate.quad`. Though for a single integral there isn't really any benefit of this,

In [3]:
res, error = quad(test_integrand_func, 0, 1, (1., 1., 1., 1., 1., 1.))

In [4]:
res, error

(1.5, 1.6653345369377348e-14)

In [5]:
res, error = scipy.integrate.quad(test_integrand_func, 0, 1, (1., 1., 1., 1., 1., 1.))

In [6]:
res, error

(1.5, 1.6653345369377348e-14)

# pyquad.quad_grid

This is the regime where pyquad is worth the investment. If you have 6 parameters in your integrand, and you want to integrate over a range of 2 of them then in scipy it may look something like this,

In [7]:
grid = np.random.random((10000000, 2))

In [8]:
%%time
res = np.zeros(grid.shape[0])
err = np.zeros(grid.shape[0])   
                                                                                   
for i in range(res.shape[0]):                                                                    
    res[i], err[i] = scipy.integrate.quad(test_integrand_func, 0, 1, 
                                          (grid[i, 0], grid[i, 1], 1.0, 1.0, 1.0, 1.0))

CPU times: user 1min 40s, sys: 62.2 ms, total: 1min 40s
Wall time: 1min 40s


In [9]:
res, err

(array([1.11591324, 1.03565151, 1.29417096, ..., 1.0791415 , 1.08966962,
        1.00045506]),
 array([1.23891258e-14, 1.14980415e-14, 1.43681840e-14, ...,
        1.19808774e-14, 1.20977630e-14, 1.11072825e-14]))

but with `pyquad.quad_grid` we are able to make the code more elegant and significantly faster by reducing calls to the python function and also compiling the integrand. All of this is done behind the API.

In [10]:
%%time
res, err = quad_grid(test_integrand_func, 0, 1, grid, (1.0, 1.0, 1.0, 1.0))

CPU times: user 1.43 s, sys: 104 ms, total: 1.53 s
Wall time: 1.53 s


In [11]:
res, err

(array([1.11591324, 1.03565151, 1.29417096, ..., 1.0791415 , 1.08966962,
        1.00045506]),
 array([1.23891258e-14, 1.14980415e-14, 1.43681840e-14, ...,
        1.19808774e-14, 1.20977630e-14, 1.11072825e-14]))

These both yield the same results, but in a slightly different time frame!

It should also be noted the number of parameters in the grid can be varied, i.e, if you have a integrand which takes 6 parameters (`test_integrand_func`) then it is possible to pass all the paramters via the grid, or just one. See the exmaples below,

In [12]:
grid1 = np.random.random((10000000, 6))
res, err = quad_grid(test_integrand_func, 0, 1, grid1)

In [13]:
grid2 = np.random.random((10000000, 1))
res, err = quad_grid(test_integrand_func, 0, 1, grid2, (1., 1., 1., 1., 1.))

# Parallel pyquad.quad_grid

If you have a lot of integrals or a nice computer then you may benefit from using openMP. Just make sure that the OMP library is available to the compiler,

In [14]:
%%time
res, err = quad_grid(test_integrand_func, 0, 1, grid, (1.0, 1.0, 1.0, 1.0), parallel=True)

CPU times: user 2.96 s, sys: 128 ms, total: 3.09 s
Wall time: 452 ms


In [15]:
res, err

(array([1.11591324, 1.03565151, 1.29417096, ..., 1.0791415 , 1.08966962,
        1.00045506]),
 array([1.23891258e-14, 1.14980415e-14, 1.43681840e-14, ...,
        1.19808774e-14, 1.20977630e-14, 1.11072825e-14]))

# What about vs. a jitted function?

It is possible to make a function faster by using numba jit to compile the function. I'm using a jit wrapper from PyAutoLens to compare to.

In [16]:
@jit_integrand
def test_integrand_func_jit(x, alpha, beta, i, j, k, l):                                             
    return x * alpha * beta + i * j * k  

then we use the naive python loop again,

In [17]:
%%time
res = np.zeros(grid.shape[0])
err = np.zeros(grid.shape[0])   
                                                                                   
for i in range(res.shape[0]):                                                                    
    res[i], err[i] = scipy.integrate.quad(test_integrand_func_jit, 0, 1, 
                                          (grid[i, 0], grid[i, 1], 1.0, 1.0, 1.0, 1.0))

CPU times: user 29.9 s, sys: 56 ms, total: 29.9 s
Wall time: 29.9 s


Which is actually pretty fast! But, if speed is critical then just using numba jit is still no where near as fast a (parallel) `grid_quad`.

If you have any questions or issues, please get in touch! (a.j.kelly@durham.ac.uk)